In [1]:
%pylab inline
import glob
import sys
import os
from collections import OrderedDict
import caffe
from DeepImageSynthesis import *

#set caculation mode and choose gpu device

gpu = 0
caffe.set_mode_gpu() #for cpu mode do 'caffe.set_mode_cpu()'
caffe.set_device(gpu)
'''
caffe.set_mode_cpu()
'''
#Path parameters
#VGGweights = '/home/yeze/Documents/gitxivprojects/DeepTextures/Models/VGG_normalised.caffemodel'
#VGGmodel = '/home/yeze/Documents/gitxivprojects/DeepTextures/Models/VGG_ave_pool_deploy.prototxt'
VGGmodel = 'Models/testmodel/cut.prototxt'
VGGweights = 'Models/testmodel/dst.caffemodel'
source_img_name = '/home/yeze/Documents/gitxivprojects/DeepTextures/Images/lishu/12.jpg'
imagenet_mean = numpy.array([ 0.40760392,  0.45795686,  0.48501961]) #mean for color channels (bgr)

Populating the interactive namespace from numpy and matplotlib


In [3]:
#load source image
im_size = 36.
source_img, net = load_resources(caffe.io.load_image(source_img_name), im_size, 
                            VGGmodel, VGGweights, imagenet_mean, 
                            show_img=True)
im_size = asarray(source_img.shape[-2:])
initpic = caffe.io.load_image('./Images/songti/129.jpg')

Image loaded!
Net loaded!


In [2]:
#l-bfgs parameters optimisation
maxiter = 50
m = 10

#define layers to include in the texture model and weights w_l
tex_layers = ['pool2', 'pool1', 'conv1']
tex_weights = [1e9,1e9,1e9]

#pass image through the network and save the constraints on each layer
constraints = OrderedDict()
net.forward(data = source_img)
for l,layer in enumerate(tex_layers):
    constraints[layer] = constraint([LossFunctions.gram_mse_loss],
                                    [{'target_gram_matrix': gram_matrix(net.blobs[layer].data),
                                      'weight': tex_weights[l]}])
#get optimisation bounds
bounds = get_bounds([source_img],im_size)

#generate new texture
result = ImageSyn(net, constraints, init=initpic, bounds=bounds,
                  callback=lambda x: show_progress(x,net), 
                  minimize_options={'maxiter': maxiter,
                                    'maxcor': m,
                                    'fttol': 0, 'gtol': 0})

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [4]:
im_size

array([36, 36])

In [6]:
len(get_bounds([source_img],im_size))

3888

In [7]:
net.blobs['data'].data.shape

(1, 3, 36, 36)